In [1]:
import xarray
import mule

In [2]:
ds = xarray.open_dataset('/g/data/w35/saw562/access-esm/ozone/intermediate/mmro3_zlev_zonal_esm.nc')

In [3]:
ds

<xarray.Dataset>
Dimensions:  (lat: 145, time: 13212, z: 38)
Coordinates:
  * time     (time) object 0750-01-15 00:00:00 ... 1850-12-15 00:00:00
  * z        (z) float64 20.0 80.0 180.0 320.0 ... 2.922e+04 3.291e+04 3.925e+04
  * lat      (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
Data variables:
    mmro3    (time, z, lat) float64 ...
Attributes:
    history:  Fri Oct  2 16:33:50 2020: ncatted -a standard_name,z,c,c,height...
    NCO:      netCDF Operators version 4.9.2 (Homepage = http://nco.sf.net, C...

In [4]:
# Total number of output slices
ds.sizes['time'] * ds.sizes['z']

502056

In [5]:
# ESM max slices in a file is 100,000 - which is about 200 years (NANCIL_LOOKUPSA)
100000 / (ds.sizes['z']*12)

219.2982456140351

In [6]:
# So 5 files, each for 200 years
ds_pmip = ds.sel(time=slice('0850','1850'))

In [7]:
# We've got 1001 years (850-1850), keep the first separate for spinup
for k,v in ds_pmip.resample(time='200Y', closed='right'):
    print(v.time.values[0], v.time.values[-1])

0850-01-15 00:00:00 0850-12-15 00:00:00
0851-01-15 00:00:00 1050-12-15 00:00:00
1051-01-15 00:00:00 1250-12-15 00:00:00
1251-01-15 00:00:00 1450-12-15 00:00:00
1451-01-15 00:00:00 1650-12-15 00:00:00
1651-01-15 00:00:00 1850-12-15 00:00:00


In [8]:
# Source ancil file
anc_in = mule.AncilFile.from_file('/g/data/access/payu/access-esm/input/historical/atmosphere/ozone_1849_2015_ESM1.anc')

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.10/lib/python3.8/site-packages/mule/stashmaster.py:284: UserWarning: Ancillary files do not define the UM version number in the Fixed Length Header. No STASHmaster file loaded: Fields will not have STASH entries attached.
  warnings.warn(msg)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.10/lib/python3.8/site-packages/mule/validators.py:191: UserWarning: 
File: /g/data/access/payu/access-esm/input/historical/atmosphere/ozone_1849_2015_ESM1.anc
Ancillary file contains header components other than the row/column dependent constants - these should be set to "None" for Ancillary files
  warnings.warn(msg)


In [9]:
class CopyXarrayOp(mule.DataOperator):
    def __init__(self):
        pass
    
    def new_field(self, source):
        source_field, source_da = source
        field = source_field.copy()
        
        field.lbyr = source_da.time.dt.year.values
        field.lbmon = source_da.time.dt.month.values
        field.lbdat = source_da.time.dt.day.values
        field.lbday = source_da.time.dt.dayofyear.values
        
        field.lbyrd = source_da.time.dt.year.values
        field.lbmond = source_da.time.dt.month.values
        field.lbdatd = source_da.time.dt.day.values
        field.lbdayd = source_da.time.dt.dayofyear.values
        
        return field
    
    def transform(self, source, new_field):
        source_field, source_da = source
        return source_da.data

In [10]:
def create_anc(anc_in, da):
    op = CopyXarrayOp()
    
    anc_out = anc_in.copy()
    anc_out.level_dependent_constants = None
    
    anc_out.fixed_length_header.t1_year = da.time.dt.year.values[0]
    anc_out.fixed_length_header.t1_month = da.time.dt.month.values[0]
    anc_out.fixed_length_header.t1_day = da.time.dt.day.values[0]
    anc_out.fixed_length_header.t1_year_day_number = da.time.dt.dayofyear.values[0]
    
    anc_out.fixed_length_header.t2_year = da.time.dt.year.values[-1]
    anc_out.fixed_length_header.t2_month = da.time.dt.month.values[-1]
    anc_out.fixed_length_header.t2_day = da.time.dt.day.values[-1]
    anc_out.fixed_length_header.t2_year_day_number = da.time.dt.dayofyear.values[-1]
    
    anc_out.integer_constants.num_times = da.sizes['time']
    
    for t in range(da.sizes['time']):
        for z in range(da.sizes['z']):
            
            anc_out.fields.append(op([anc_in.fields[z], da.isel(time=t, z=z)]))
            
    return anc_out

In [11]:
anc_out = create_anc(anc_in, ds.mmro3.sel(time='0850'))

In [12]:
anc_out.to_file('ozone_esm_pmip_0850-0850.anc')

In [13]:
for k,v in ds_pmip.resample(time='200Y', closed='right'):
    file = f"ozone_esm_pmip_{v.time.dt.year.values[0]:04d}-{v.time.dt.year.values[-1]:04d}.anc"
    
    anc_out = create_anc(anc_in, v.mmro3)

    anc_out.to_file(file)
    print(file)

ozone_esm_pmip_0850-0850.anc
ozone_esm_pmip_0851-1050.anc
ozone_esm_pmip_1051-1250.anc
ozone_esm_pmip_1251-1450.anc
ozone_esm_pmip_1451-1650.anc
ozone_esm_pmip_1651-1850.anc


In [14]:
ds.time.dt.dayofyear

<xarray.DataArray 'dayofyear' (time: 13212)>
array([ 15,  45,  75, ..., 285, 315, 345])
Coordinates:
  * time     (time) object 0750-01-15 00:00:00 ... 1850-12-15 00:00:00